In [1]:
import numpy as np
import pandas as pd
import jieba
import string
from fastText import load_model
import platform
platform.system()

'Darwin'

In [2]:
cwe = np.load("zh.bin.syn0.npy")

In [4]:
cwe.shape

(50101, 300)

In [5]:
cwe_neg = np.load("zh.bin.syn1neg.npy")

In [6]:
cwe_neg.shape

(50101, 300)

# Word Embedding Matrix Lookup (cross platform)
Reference: [https://github.com/facebookresearch/fastText/tree/master/python](https://github.com/facebookresearch/fastText/tree/master/python)

In [2]:
if platform.system() == 'Windows':
    with open("zh/wiki.zh.vec", "r", encoding='utf-8') as fin:
        words = fin.readlines()
elif platform.system() == 'Darwin':  # Mac OSX
    with open("zh/wiki.zh.vec", "r") as fin:
        words = fin.readlines()

In [3]:
len(words)

332648

In [4]:
if platform.system() == 'Windows':
    wiki_vocab = [x.split(' ')[0] for x in words[1:]]
elif platform.system() == 'Darwin':
    wiki_vocab = [x.split(' ')[0].decode('utf-8') for x in words[1:]]

In [5]:
len(wiki_vocab)

332647

In [6]:
print(wiki_vocab[100])

日本


In [ ]:
# wiki_embeddings = np.array([x.strip().split(' ')[1:] for x in words[1:]])  # will probably run out of memory

In [8]:
def read_vocab(file_path):
    if platform.system() == 'Windows':
        with open(file_path, 'r', encoding='utf-8') as fin:
            vocab = [x.strip() for x in fin.readlines()]
            vocab = [x for x in vocab if x != '']
    elif platform.system() == 'Darwin':
        with open(file_path, 'r') as fin:
            vocab = [x.strip() for x in fin.readlines()]
            vocab = [x.decode('utf-8') for x in vocab if x != '']
    return vocab

In [9]:
vocab = read_vocab('../01_WebScraping/vocab.txt')

In [10]:
len(vocab)

9827

In [11]:
print(vocab[0])

沉静


In [2]:
def custom_tokenizer(text):
    return [token for token in jieba.cut(text) if token not in string.punctuation+" " and token != '\n']

In [3]:
def read_lyrics(file_path):
    if platform.system() == 'Windows':
        lyrics = [line.strip() for line in open(file_path, 'r', encoding='utf-8').readlines()]
        lyrics = [custom_tokenizer(text) for text in lyrics]
    elif platform.system() == 'Darwin':
        lyrics = [line.strip() for line in open(file_path, 'r').readlines()]
        lyrics = [custom_tokenizer(text) for text in lyrics]
    return lyrics

In [4]:
lyrics = read_lyrics('../01_WebScraping/lyrics.txt')

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/45/9s5cv4kn143fggkxy4bdc89c0000gn/T/jieba.cache
DEBUG:jieba:Dumping model to file cache /var/folders/45/9s5cv4kn143fggkxy4bdc89c0000gn/T/jieba.cache
Loading model cost 3.568 seconds.
DEBUG:jieba:Loading model cost 3.568 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [5]:
for item in lyrics[:3]:
    print(len(item))

82
210
161


# Token look up

In [15]:
def token_lookup(token, vocab, word_embedding):
    try:
        return [float(x) for x in word_embedding[vocab.index(token)+1].strip().split(' ')[1:]]
    except ValueError:
        print('Token not in vocabulary!')
        # See https://github.com/facebookresearch/fastText#obtaining-word-vectors-for-out-of-vocabulary-words for more details
        return ''

In [16]:
len(token_lookup(vocab[5], wiki_vocab, words))

300

In [12]:
# Skipgram model
model = load_model('zh/wiki.zh.bin')

In [13]:
print(model.get_word_vector(vocab[0]))

[-0.14655262 -0.35390663  0.2001343  -0.088716    0.09892082 -0.02019173
 -0.2750019  -0.03976827  0.10884193 -0.1401302  -0.3022368  -0.32495514
  0.043915    0.08580057  0.42206717 -0.2760704   0.15125434 -0.11696516
  0.2907221  -0.41516507 -0.08458394  0.20616367  0.12262227  0.14075808
  0.06585976 -0.11330824 -0.61613953 -0.19981228  0.11573618  0.26924527
 -0.3895331   0.02170827  0.18145365  0.395527    0.08334675  0.24473996
 -0.06434914  0.08435744  0.50620776 -0.02277536 -0.00557652  0.33351102
 -0.2400871   0.17134102 -0.12819333  0.14156206 -0.42642173 -0.1798166
 -0.21311565 -0.11199573  0.00104835 -0.38445362 -0.01106974 -0.33570626
 -0.21296307  0.0680915  -0.44446182 -0.14636514  0.46745524  0.06082931
 -0.28635907 -0.03316678  0.2592      0.18856986 -0.43816504 -0.4466467
  0.12617154 -0.04868822  0.108766   -0.20478442 -0.10747581  0.06855971
 -0.30118525 -0.25657704  0.30651236  0.04430697 -0.01705025 -0.0157372
  0.19639769  0.08805114 -0.0461695  -0.06703023 -0.26